# Context for the LLM

I used an "official" LLM rather than an uncensored because of health being involved. I woudln't want our pseudo-60-year-old beginner runner to be discouraged when the model tells him he can't run, or worse, tells him he can run at a 6:00 mi/hr pace if he just starts TODAY. That would injure the people we're trying to help advise. That's not my goal with **Motiva**.

I made a lot of iterations to the code I have now. And I mean **A LOT.** Some of the iterations have been deleted because they've failed to compile, others were just duplicates of what I had before. I'll try my best to explain the movement and growth of the code as best as I can.

Overall, the general idea followed like this: </br>

Chatbox to talk to about running **->** Chatbox to talk to about running with more preliminary question asking to make it more user friendly **->** Chatbox to talk to about running and be able to look online for reference **->** Chatbox to talk to about running AND you can upload a CSV file to get a running summary overview to reference

---

## 1. Very Preliminary, First Ideas

In [ ]:
run_coach/
├── app.py                 # Main app file (Streamlit or Gradio)
├── chatbot.py             # Chat logic and LLM prompt handling
├── strava_api.py          # Strava OAuth + data fetching
├── user_data.py           # User profile and local data store
├── planner.py             # Running plan generator
└── utils.py               # Helper functions

**Core Chat Flow (Duet-style interaction)**

In [ ]:
# chatbot.py (simplified example)
def handle_user_message(message, user_state):
    if not user_state.get("goal"):
        return "What's your running goal? 3K, 5K, half marathon, etc."
    
    if not user_state.get("runner_type"):
        return "What kind of runner are you? Beginner, casual, or expert?"
    
    if not user_state.get("shoe_mileage"):
        return "What shoes do you run with, and how many miles are on them?"
    
    if user_state["shoe_mileage"] > 500:
        return "Looks like your shoes have over 500 miles—you might want to replace them."

    if not user_state.get("issues"):
        return "Do you have any recurring issues while running? (shin splints, knees, hips...)"
    
    if not user_state.get("strava_connected"):
        return "Would you like to connect your Strava account to sync your running data?"
    
    return "What would you like to start with today? (Create a running plan, shoe recommendation, injury support)"

This helped with keeping in check that the model would ask preliminary questions to understand the user before it jumped into overloadig the user with suggestions.

**Create strava_api.py** :
I wanted to integrated strava in the code, but I'm not an expert coder so later on I scrapped it because it was too complicated.

In [ ]:
import os
import requests
from dotenv import load_dotenv

load_dotenv()

STRAVA_CLIENT_ID = os.getenv("STRAVA_CLIENT_ID")
STRAVA_CLIENT_SECRET = os.getenv("STRAVA_CLIENT_SECRET")
REDIRECT_URI = os.getenv("STRAVA_REDIRECT_URI")  # e.g. http://localhost:8000/exchange_token

def get_strava_auth_url():
    return f"https://www.strava.com/oauth/authorize?client_id={STRAVA_CLIENT_ID}&response_type=code&redirect_uri={REDIRECT_URI}&approval_prompt=auto&scope=read,activity:read"

def exchange_code_for_token(code):
    response = requests.post("https://www.strava.com/oauth/token", data={
        'client_id': STRAVA_CLIENT_ID,
        'client_secret': STRAVA_CLIENT_SECRET,
        'code': code,
        'grant_type': 'authorization_code'
    })
    return response.json()

def get_recent_activities(access_token, limit=5):
    headers = {'Authorization': f'Bearer {access_token}'}
    response = requests.get(f"https://www.strava.com/api/v3/athlete/activities?per_page={limit}", headers=headers)
    return response.json()

In [ ]:
# add to .env
STRAVA_CLIENT_ID=your_client_id
STRAVA_CLIENT_SECRET=your_client_secret
STRAVA_REDIRECT_URI=http://localhost:8000/exchange_token

In [ ]:
# how to use in app
from strava_api import get_strava_auth_url, exchange_code_for_token, get_recent_activities

# 1. Send user to this URL to authorize
print("Visit this URL to connect your Strava:", get_strava_auth_url())

# 2. After user grants access, Strava redirects to your redirect URI with `?code=xyz`
code = input("Paste the code from the redirect URL here:\n")

# 3. Exchange code for token
tokens = exchange_code_for_token(code)
access_token = tokens["access_token"]

# 4. Fetch recent runs
activities = get_recent_activities(access_token)
for run in activities:
    print(f"{run['name']} - {run['distance']/1000:.2f} km in {run['moving_time']//60} min")

In [ ]:
# generate a plan from strava data

def summarize_recent_runs(activities):
    summary = ""
    for a in activities:
        km = round(a['distance'] / 1000, 1)
        time_min = round(a['moving_time'] / 60)
        summary += f"{a['name']}: {km} km in {time_min} min\n"
    return summary

def generate_running_plan(user_profile, activities_summary):
    prompt = f"""
You are a running coach. Based on the user's info and their recent runs, create a 4-week training plan.

Goal: {user_profile['goal']}
Experience level: {user_profile['runner_type']}
Known injuries: {', '.join(user_profile['injuries']) if user_profile['injuries'] else 'None'}

Recent runs:
{activities_summary}

Make it progressive, clear, and safe.
"""
    return model_coach.invoke([
        coach_system_msg,
        HumanMessage(prompt)
    ]).content


**Semi Full Model From the First Iteration**

In [ ]:
from dotenv import load_dotenv
import os

from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai.chat_models import ChatOpenAI

# Load environment variables (API Key, etc.)
load_dotenv()

# Initialize model
model_coach = ChatOpenAI(temperature=0.7)

# Set the coach's personality
coach_system_msg = SystemMessage(
    '''You are a knowledgeable and friendly running coach helping a runner build the perfect plan.
You ask smart questions, provide encouragement, and personalize advice based on their goals, experience level, and running history.'''
)

# Simulated user profile (in practice, this would be built interactively or from UI inputs)
user_profile = {
    "goal": None,
    "runner_type": None,
    "shoe_mileage": None,
    "injuries": [],
    "strava_connected": False
}

# Step-by-step conversation simulation
def chat_with_coach():
    conversation = [coach_system_msg]

    # 1. Ask for running goal
    if not user_profile["goal"]:
        goal_input = input("Coach: What's your running goal? (3K, 5K, 10K, half, full, ultra)\nYou: ")
        user_profile["goal"] = goal_input
        conversation.append(HumanMessage(f"My running goal is to complete a {goal_input}."))

    # 2. Ask about runner type
    if not user_profile["runner_type"]:
        type_input = input("Coach: What kind of runner are you? (Beginner, Casual, Expert)\nYou: ")
        user_profile["runner_type"] = type_input
        conversation.append(HumanMessage(f"I'm a {type_input} runner."))

    # 3. Ask about shoes
    if not user_profile["shoe_mileage"]:
        shoe_input = input("Coach: What shoes do you currently run in, and approximately how many miles are on them?\nYou: ")
        user_profile["shoe_mileage"] = int([int(s) for s in shoe_input.split() if s.isdigit()][0])
        conversation.append(HumanMessage(f"My shoes have around {user_profile['shoe_mileage']} miles on them."))

        if user_profile["shoe_mileage"] >= 500:
            conversation.append(HumanMessage("Do I need to replace my shoes soon?"))

    # 4. Injury check
    injury_input = input("Coach: Do you experience any pain or issues when running? (e.g. shin splints, knee pain, hip soreness)\nYou: ")
    if injury_input.lower() != "no":
        user_profile["injuries"] = [inj.strip() for inj in injury_input.split(",")]
        conversation.append(HumanMessage(f"I sometimes experience: {injury_input}"))

    # 5. Strava connection placeholder
    strava_input = input("Coach: Would you like to connect your Strava account for personalized training based on your recent runs? (yes/no)\nYou: ")
    user_profile["strava_connected"] = strava_input.lower() == "yes"
    if user_profile["strava_connected"]:
        conversation.append(HumanMessage("Yes, I’d like to connect my Strava account."))

    # 6. Ask what to start with
    next_step = input("Coach: What would you like to start with today? (Create a running plan, Find new shoes, Address running pain)\nYou: ")
    conversation.append(HumanMessage(f"I'd like to start with: {next_step}"))

    # Invoke model with the full conversation
    coach_response = model_coach.invoke(conversation)

    print("\n🏃‍♂️ Coach Response:\n", coach_response.content)


if __name__ == "__main__":
    chat_with_coach()

---

## 2. Second Iteration That Scraps Strava API

In [ ]:
# running_coach_chatbot.py (Strava-free version)

import os
from dotenv import load_dotenv
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai.chat_models import ChatOpenAI

# Load environment variables
load_dotenv()

# Initialize ChatGPT model
model_coach = ChatOpenAI(temperature=0.9)

# System message for coach
coach_system_msg = SystemMessage(
    '''You are a knowledgeable and friendly running coach helping a runner build the perfect plan for their goals.
You ask smart questions, provide encouragement, and personalize advice based on their goals, experience level, and running history.'''
)

# Simulated user profile
user_profile = {
    "goal": None,
    "runner_type": None,
    "shoe_mileage": None,
    "injuries": []
}

def generate_running_plan(user_profile, activity_summary):
    prompt = f"""
You are a running coach. Based on the user's info and their recent runs, create a training plan based on how many weeks they want to train. 
Ask them how long they want to train for (weeks, months, year). From there on, adjust and modify their plans.

Goal: {user_profile['goal']}
Experience level: {user_profile['runner_type']}
Known injuries: {', '.join(user_profile['injuries']) if user_profile['injuries'] else 'None'}

User's self-reported recent activity:
{activity_summary}

Make it progressive, clear, and safe.
"""
    return model_coach.invoke([
        coach_system_msg,
        HumanMessage(prompt)
    ]).content

def chat_with_coach():
    conversation = [coach_system_msg]

    # 1. Ask for running goal
    if not user_profile["goal"]:
        goal_input = input("Coach: What's your running goal? (3K, 5K, 10K, half, full, ultra)\nYou: ")
        user_profile["goal"] = goal_input
        conversation.append(HumanMessage(f"My running goal is to complete a {goal_input}."))

    # 2. Ask about runner type
    if not user_profile["runner_type"]:
        type_input = input("Coach: What kind of runner are you? (Beginner, Casual, Expert)\nYou: ")
        user_profile["runner_type"] = type_input
        conversation.append(HumanMessage(f"I'm a {type_input} runner."))

    # 3. Ask about shoes
    if not user_profile["shoe_mileage"]:
        shoe_input = input("Coach: What shoes do you currently run in, and approximately how many miles are on them?\nYou: ")
        user_profile["shoe_mileage"] = int([int(s) for s in shoe_input.split() if s.isdigit()][0])
        conversation.append(HumanMessage(f"My shoes have around {user_profile['shoe_mileage']} miles on them."))
        if user_profile["shoe_mileage"] >= 500:
            conversation.append(HumanMessage("Do I need to replace my shoes soon?"))

    # 4. Injury check
    injury_input = input("Coach: Do you experience any pain or issues when running? (e.g. shin splints, knee pain, hip soreness)\nYou: ")
    if injury_input.lower() != "no":
        user_profile["injuries"] = [inj.strip() for inj in injury_input.split(",")]
        conversation.append(HumanMessage(f"I sometimes experience: {injury_input}"))

    # 5. User self-reporting recent activity
    activity_summary = input("Coach: Can you describe your recent running activity? (e.g. 3 runs per week, longest run was 5 miles)\nYou: ")
    conversation.append(HumanMessage(f"My recent running activity: {activity_summary}"))

    # 6. Ask what to start with
    next_step = input("Coach: What would you like to start with today? (Create a running plan, Find new shoes, Address running pain)\nYou: ")
    conversation.append(HumanMessage(f"I'd like to start with: {next_step}"))

    # 7. Generate plan if applicable
    if "plan" in next_step.lower():
        plan = generate_running_plan(user_profile, activity_summary)
        print("\n🏃‍♂️ Coach Training Plan:\n", plan)
        conversation.append(HumanMessage(plan))
    else:
        coach_response = model_coach.invoke(conversation)
        print("\n🏃‍♂️ Coach Response:\n", coach_response.content)
        conversation.append(coach_response)

    # 🔁 Loop for ongoing conversation
    while True:
        user_input = input("\nYou: ")
        if user_input.lower() in ["exit", "quit", "bye"]:
            print("Coach: Keep running strong! Talk to you soon 🏃‍♀️💨")
            break

        conversation.append(HumanMessage(user_input))
        coach_response = model_coach.invoke(conversation)
        print("\nCoach:", coach_response.content)
        conversation.append(coach_response)

if __name__ == "__main__":
    chat_with_coach()

**The problem with this** code was that it didn't have a good user interface and you were chatting with the AI in a loop. Also, I couldn't really upload files at this point. Moreover, it couldn't reference sources outside to help me run better. But this was a great elementary model; it was able to answer any questions I had about running.

---

## 3. Gradio for Better User Experience

In [ ]:
import os
import gradio as gr
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage

# Load .env
load_dotenv()

# Setup model
model_coach = ChatOpenAI(model="gpt-4o", temperature=0.9)

# System message
coach_system_msg = SystemMessage(
    """You are a knowledgeable and friendly running coach helping a runner build the perfect plan for their goals.
You ask smart questions, provide encouragement, and personalize advice based on their goals, experience level, and running history."""
)

# User profile and conversation state
user_profile = {
    "goal": None,
    "runner_type": None,
    "shoe_mileage": None,
    "injuries": []
}
conversation = [coach_system_msg]
step_index = 0
activity_summary = ""

# Step-by-step guided function
def guided_chat(user_input, history):
    global step_index, user_profile, activity_summary, conversation

    # Process answers to previous step
    if step_index == 1:
        user_profile["goal"] = user_input
        conversation.append(HumanMessage(f"My running goal is to complete a {user_input}."))
    elif step_index == 2:
        user_profile["runner_type"] = user_input
        conversation.append(HumanMessage(f"I'm a {user_input} runner."))
    elif step_index == 3:
        if any(char.isdigit() for char in user_input):
            miles = int([int(s) for s in user_input.split() if s.isdigit()][0])
            user_profile["shoe_mileage"] = miles
            conversation.append(HumanMessage(f"My shoes have around {miles} miles on them."))
            if miles >= 500:
                conversation.append(HumanMessage("Do I need to replace my shoes soon?"))
    elif step_index == 4:
        if user_input.lower() != "no":
            user_profile["injuries"] = [inj.strip() for inj in user_input.split(",")]
            conversation.append(HumanMessage(f"I sometimes experience: {user_input}"))
    elif step_index == 5:
        activity_summary = user_input
        conversation.append(HumanMessage(f"My recent running activity: {activity_summary}"))
    elif step_index == 6:
        conversation.append(HumanMessage(f"I'd like to start with: {user_input}"))
        if "plan" in user_input.lower():
            prompt = f"""
You are a running coach. Based on the user's info and their recent runs, create a training plan.
Goal: {user_profile['goal']}
Experience level: {user_profile['runner_type']}
Known injuries: {', '.join(user_profile['injuries']) or 'None'}
User's recent activity: {activity_summary}
Make it progressive, clear, and safe.
"""
            response = model_coach.invoke([coach_system_msg, HumanMessage(prompt)])
            conversation.append(response)
            return response.content
        else:
            response = model_coach.invoke(conversation)
            conversation.append(response)
            return response.content

    # Advance to next step
    step_index += 1

    # Ask next question
    questions = [
        "Coach: What's your running goal? (3K, 5K, 10K, half, full, ultra)",
        "Coach: What kind of runner are you? (Beginner, Casual, Expert)",
        "Coach: What shoes do you currently run in, and approximately how many miles are on them?",
        "Coach: Do you experience any pain or issues when running? (e.g. shin splints, knee pain, hip soreness)",
        "Coach: Can you describe your recent running activity? (e.g. 3 runs per week, longest run was 5 miles)",
        "Coach: What would you like to start with today? (Create a running plan, Find new shoes, Address running pain)"
    ]

    if step_index < len(questions) + 1:
        return questions[step_index - 1]
    else:
        return "You're all set! Feel free to ask me anything else."

# Launch Gradio chat
gr.ChatInterface(
    fn=guided_chat,
    type="messages",
    title="🏃‍♂️ Bob the Running Coach",
    description="Bob will guide you step-by-step to build your perfect running plan."
).launch(share=True)


**It was better on the user experience side**, but I wanted the model to be more helpful for people who had Strava Data to upload and get a summary of their progress. Moreover, I still wanted my model to be able to look at other references for me to do deeper research.

---

## 4. A lot of Debugging and moving pieces around

---

## 5. Final Version of Motiva

![Motiva, an AI running coach](MOTIVA_LOGO.png)

In [ ]:
import os
import gradio as gr
import json
import pandas as pd
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_community.tools import DuckDuckGoSearchRun

# Load .env
load_dotenv()

# Setup models and tools
llm = ChatOpenAI(model="gpt-4o")
search_tool = DuckDuckGoSearchRun()
llm_with_tools = llm.bind_tools([search_tool])

# Define prompt structure
prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a knowledgeable and friendly running coach. 
Step 1: Introduce yourself to the user as Motiva, an AI running coach. Tell them that you are here to help them with their running goals and teach them more about running. You should always wait for a response from the user after every question before moving on. Work on understanding their goals. Ask one question at a time and explain that you are asking these questions to personalize the chat. Do not start explaining right away before gathering their information.

Step 2: Look up information that would help you answer their questions and complete their goals. They might offer a running summary of their progress, and to that assess whether their goals are possible. Think step by step and make a plan based on the goal. Once you know about the runner and their goals, advise the runner in an informed, helpful way. Break down the advice and tailor your response and questions to the runner. This will change as the conversation progresses. Once you have enough information, tell the runner you are happy to help them with other things they need help with.

Step 3: Close the conversation. When the runner has no more questions or doesn't need any more help, please insert a motivational quote and tell them you are here to help in the future."""),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
])
chain = prompt | llm_with_tools | StrOutputParser()

# Simulated user profile (persistent between messages)
user_profile = {
    "goal": None,
    "runner_type": None,
    "shoe_mileage": None,
    "injuries": []
}

# Store uploaded running data
running_data_df = None

# Gradio-compatible chat function
def chat_fn(message, history):
    global running_data_df

    # Preprocess input to simulate memory update
    if "goal" in message.lower() and not user_profile["goal"]:
        user_profile["goal"] = message
    if "beginner" in message.lower() or "casual" in message.lower() or "expert" in message.lower():
        user_profile["runner_type"] = message
    if "miles" in message.lower() and any(char.isdigit() for char in message):
        user_profile["shoe_mileage"] = int([int(s) for s in message.split() if s.isdigit()][0])
    if "pain" in message.lower() or "injury" in message.lower():
        user_profile["injuries"].append(message)

    print(f"USER PROFILE: {user_profile}")  # Debug print

    # Inject summary into prompt context if running data is available
    if running_data_df is not None:
        summary = generate_summary(running_data_df)
        message = f"{summary}\n\nUser says: {message}"

    response = chain.invoke({"input": message, "chat_history": history})
    return response

# File upload function
def handle_file(file):
    global running_data_df
    running_data_df = pd.read_csv(file.name)
    print(running_data_df.columns)  # Debug print to inspect column names
    return generate_summary(running_data_df)

# Generate quick summary
def generate_summary(df):
    try:
        df = df.copy()
        total_distance = df['distance'].sum() / 1000  # Convert to km
        avg_distance = df['distance'].mean() / 1000  # Convert to km

        # Filter valid pace entries (distance > 0)
        valid_df = df[(df['distance'] > 0) & (df['moving_time'] > 0)]

        if not valid_df.empty:
            valid_df['pace_min_per_km'] = (valid_df['moving_time'] / valid_df['distance']) * (1000 / 60)
            avg_pace = valid_df['pace_min_per_km'].mean()
        else:
            avg_pace = "N/A"

        total_runs = len(df)

        if isinstance(avg_pace, float) and not pd.isna(avg_pace):
            pace_str = f"{avg_pace:.2f} min/km"
        else:
            pace_str = str(avg_pace)

        summary = f"""\n📊 **Running Data Summary**:
- Total runs: {total_runs}
- Total distance: {total_distance:.2f} km
- Average distance: {avg_distance:.2f} km/run
- Average pace: {pace_str}"""

        return summary
    except Exception as e:
        return f"Error reading file: {e}"

# Launch Gradio interface
with gr.Blocks() as demo:
    gr.Markdown("# 🏃 Running Coach Chatbox")
    file_input = gr.File(label="Upload CSV Running Data", file_types=[".csv"])
    file_output = gr.Textbox(label="Data Summary", lines=6)

    file_input.change(fn=handle_file, inputs=file_input, outputs=file_output)

    gr.ChatInterface(fn=chat_fn, type="messages")

demo.launch(share=True)


**Motiva** final iteration. This version is able to summarize your running data after you submit a CSV file. You can copy the summary and paste it into the chatbox to figure out your goals. If you ask questions like "what sites are good for me to base my running plan from" it helps find them for you.

## Let's conclude with evaluations 

- [Evaluations](/5_Evaluations.ipynb)